In [4]:
!pip install roboflow
!pip install ultralytics

from roboflow import Roboflow
from ultralytics import YOLO

rf = Roboflow(api_key="GLsYJXT4LCmcgLDjkRev")
project = rf.workspace("shone").project("aerial-qjpyp-hs4x5")
version = project.version(3)
dataset = version.download("yolov8")

model = YOLO("yolov8n.pt")
model.train(data=dataset.location + "/data.yaml", epochs=30, imgsz=640, batch=16)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
loading Roboflow workspace...
loading Roboflow project...
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/aerial-3/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0,

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79894b5472f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [5]:
# inference_and_counting.py  (can run in notebook or terminal)
from ultralytics import YOLO
import os, csv, time
from pathlib import Path

# ---------- CONFIG ----------
MODEL_PATH = "runs/detect/train/weights/best.pt"  # adjust
IMAGES_DIR = "aerial-3/test/images"   # folder of images or folder of frames
OUT_DIR = "runs/infer_counts"
CONF_THRESH = 0.4
SAVE_ANN = True   # save annotated images
# ----------------------------

os.makedirs(OUT_DIR, exist_ok=True)
model = YOLO(MODEL_PATH)

# class names from model
names = model.model.names  # dict {0: 'car', 1: 'bike', ...}

csv_path = os.path.join(OUT_DIR, "counts_per_image.csv")
with open(csv_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    header = ["image"] + [names[i] for i in sorted(names.keys())] + ["total"]
    writer.writerow(header)

    for img_file in sorted(Path(IMAGES_DIR).glob("*.jpg")):
        results = model.predict(source=str(img_file), conf=CONF_THRESH, save=SAVE_ANN)
        # results is a list; we ran on a single image, so take first
        r = results[0]
        boxes = r.boxes  # Boxes object
        counts = {i:0 for i in names.keys()}
        for box in boxes:
            cls = int(box.cls.cpu().numpy()[0])
            counts[cls] += 1
        row = [str(img_file.name)] + [counts[i] for i in sorted(names.keys())] + [sum(counts.values())]
        writer.writerow(row)

print("CSV saved to:", csv_path)



image 1/1 /content/aerial-3/test/images/0000036_03591_d_0000048_jpg.rf.4c6fe159af85356fb568ac271138e01c.jpg: 384x640 33 persons, 40.7ms
Speed: 2.7ms preprocess, 40.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/runs/detect/predict

image 1/1 /content/aerial-3/test/images/0000047_01000_d_0000090_jpg.rf.487eb101504586319695fe2911e55e75.jpg: 384x640 49 persons, 6.5ms
Speed: 1.9ms preprocess, 6.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/runs/detect/predict

image 1/1 /content/aerial-3/test/images/0000068_01500_d_0000005_jpg.rf.9ce23683c294f5ed27f417eab440df59.jpg: 384x640 55 persons, 8.1ms
Speed: 2.0ms preprocess, 8.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/runs/detect/predict

image 1/1 /content/aerial-3/test/images/0000072_00000_d_0000001_jpg.rf.82891ad72fa4d72198215cf68fb1b733.jpg: 384x640 3 cars, 2 motorcycles, 57 persons, 6.1ms
Speed

In [8]:
import time
from ultralytics import YOLO
import glob

# Load your trained model
model = YOLO("runs/detect/train/weights/best.pt")

# Get 10–20 test images
images = glob.glob("aerial-3/test/images/*.jpg")[:20]

start = time.time()
results = model.predict(source=images, conf=0.4, verbose=False)
end = time.time()

num_images = len(images)
fps = num_images / (end - start)
print(f"\nProcessed {num_images} images in {(end - start):.2f} seconds")
print(f"Average FPS: {fps:.2f}")




Processed 20 images in 0.98 seconds
Average FPS: 20.43


In [9]:
!nvidia-smi


Sat Oct 25 13:46:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P0             28W /   70W |     962MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
import pandas as pd

# Load your CSV (change path if needed)
df = pd.read_csv("runs/infer_counts/counts_per_image.csv")

# If your CSV already has a 'total' column with total objects per image:
if "total" in df.columns:
    totals = df["total"]

# If not, we'll just sum all numeric columns (like car, bike, person, animal)
else:
    totals = df.select_dtypes("number").sum(axis=1)

avg_objs = totals.mean()
median_objs = totals.median()
max_objs = totals.max()

print(f"Images: {len(df)}")
print(f"Average objects per frame: {avg_objs:.2f}")
print(f"Median objects per frame: {median_objs}")
print(f"Maximum objects in one frame: {max_objs}")


Images: 476
Average objects per frame: 15.09
Median objects per frame: 10.0
Maximum objects in one frame: 101
